## **Post-Training Static Quantization**

Static quantization은 각 층의 Activation에 대한 clipping range가 input tensor값에 상관없이 고정(fixed, static)되어 있는 경우이다.

입력값(예를 들어, 입력 이미지)에 따라서 각 층의 activation값이 변화하므로 sampling data를 이용해서 각 층의 activation에 대한 clipping range에 대한 수치를 수집한 후에, 최빈값(가장 빈도수가 많은 값)으로 quantization parameter를 정하게 된다.

PyTorch의 경우, 딥러닝 모델에서 각 층의 activation값을 추출하는 것은 register_forward_hook이라는 방법을 이용한다.

예를 들어, 100개의 서로 다른 입력값에 대해서 모델의 각 층의 activation값과 이에 따른 quantization parameter(scaling factor)를 구하고, quantization parameter 중에서 최빈도값을 최종 quantization parameter로 결정한 후에, 이 값을 이용해서 모델의 각 층의 activation값을 양자화하게 된다.

이 경우에 입력값에 따라 clipping range를 보정하는 과정이 딥러닝 모델의 수행 과정 중에 포함되지 않는 장점이 있지만, 입력값에 따라 정해진 clipping range가 맞지 않을 경우에는 모델의 예측값이 맞지 않을 가능성이 있다.

### **Import Packages**

In [ ]:
import os
#import pickle
#import time
#import datetime
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
# import torch.optim as optim

import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
#check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available. Training on CPU ...')
else:
    print('CUDA is available! Training on GPU ...')

### **Functions**

In [ ]:
#output: model output
#target: correct answer
#topk: number of k best model output

def accuracy(output, target ,topk=(1,)):
  """
  Computes the accuracy over the k top predictions for the specified values of k
  IN top-5 accuracy you give yourself credit for having the right answer
  if the right answer appears in your top five guesses
  """
  with torch.no_grad(): #no need for gradient
    maxk = max(topk) #store biggest topk value
    batch_size=target.size(0) #check batch size // express "target" tensor's 0th dimension size

    _, pred = output.topk(maxk,1,True,True) #maxk=bring upper k value / dim=selected class dimension / 1st True=largest, if smallest use False / 2nd True=sort result? yes=True, no=False
    pred = pred.t() #transpose tensor pred

    #correct = pred.eq(target. view(1, -1).expand_as(pred))
    #correct = (pred == target.view(1, -1).expand_as(pred))
    correct = (pred == target.unsqueeze(dim=0)).expand_as(pred)

    res = []
    for k in topk:
      #correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
      correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
      res.append(correct_k.mul(1.0/batch_size))
    return res

class AverageMeter(object):
  """Computes and stores the average and current value"""

  def __init__(self, name, fmt=':f'): #floating point format
    self.name = name
    self.fmt = fmt
    self.reset()

  def reset(self):
    self.val=0
    self.avg=0
    self.sum=0
    self.count=0

  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.num / self.count

  def __str__(self):
    #fmtstr = '{name} {val' + self.fmt + '}({avg' + self.fmt+ '})'
    fmtstr = '{name} ({avg' + self.fmt +  '})'

    return fmtstr.format(**self.__dict__)

def norm1(X,Y):
  return np.sum(np.abs(X-Y))

def norm2(X,Y):
  return np.sqrt(np.sum(np.square(X-Y)))

def quantize(X, NBIT=8):
    #symmetry quantization, per tensor
    #1. find threshold
    threshold = np.max([np.abs(np.max(X)), np.abs(np.min(X))])

    QRANGE = 2**(NBIT-1)

    #2. find p: decimal position of quantized value
    p = np.int(np.log2((QRANGE-1)/threshold))

    #3. quantize using 8 bit
    #3.1 apply threshold
    data_th = np.clip(X, -QRANGE*2**(-p), (QRANGE-1)*2**(-p))

    #3.2 calculate the scale factor for quantization
    SCALE = 2**(-p)

    #3.3 quantize (apply scale factor)
    #we are using a rounding function to force the quantized values to be the whole numbers which INT8 can represent
    data_qn = np.round(data_th/SCALE)

    #3.4 dequantize (simply reverse the quantization)
    data_dqn = data_qn *SCALE
    return data_dqn, p


### **Define Dynamic Quantized Model after activation**

needs torch version >=1.7.0

In [ ]:
class Darknet19(nn.Module): # formal inheritance method in PyTorch / inherit Parent class nn.Module
  def __init__(self, num_classes: int = 1000): # 1000 is number of classes for neuralnet model to predict. It is default. you can reset it when calling the function
    super(Darknet19, self).__init__() # super(child class name, self).__init__()

    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False) #3*3 require zero padding / 1*1 does not require zero padding
    self.batchnorm1 = nn.BatchNorm2d(32) #32개 channel 출력값을 normalization, Z function(평균, 분산)-->(0,1) gradient vanishing, exploding을 방지
    self.leaky_relu1 = nn.LeakyReLU(negative_slope=0.1) #Relu negative slope

    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm2 = nn.BatchNorm2d(64)
    self.leaky_relu2 = nn.LeakyReLU(negative_slope=0.1)

    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm3 = nn.BatchNorm2d(128)
    self.leaky_relu3 = nn.LeakyReLU(negative_slope=0.1)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm4 = nn.BatchNorm2d(64)
    self.leaky_relu4 = nn.LeakyReLU(negative_slope=0.1)
    self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm5 = nn.BatchNorm2d(128)
    self.leaky_relu5 = nn.LeakyReLU(negative_slope=0.1)

    self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm6 = nn.BatchNorm2d(256)
    self.leaky_relu6 = nn.LeakyReLU(negative_slope=0.1)
    self.conv7 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm7 = nn.BatchNorm2d(128)
    self.leaky_relu7 = nn.LeakyReLU(negative_slope=0.1)
    self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm8 = nn.BatchNorm2d(256)
    self.leaky_relu8 = nn.LeakyReLU(negative_slope=0.1)

    self.conv9 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm9 = nn.BatchNorm2d(512)
    self.leaky_relu9 = nn.LeakyReLU(negative_slope=0.1)
    self.conv10 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm10 = nn.BatchNorm2d(256)
    self.leaky_relu10 = nn.LeakyReLU(negative_slope=0.1)
    self.conv11 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm11 = nn.BatchNorm2d(512)
    self.leaky_relu11 = nn.LeakyReLU(negative_slope=0.1)
    self.conv12 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm12 = nn.BatchNorm2d(256)
    self.leaky_relu12 = nn.LeakyReLU(negative_slope=0.1)
    self.conv13 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm13 = nn.BatchNorm2d(512)
    self.leaky_relu13 = nn.LeakyReLU(negative_slope=0.1)

    self.conv14 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm14 = nn.BatchNorm2d(1024)
    self.leaky_relu14 = nn.LeakyReLU(negative_slope=0.1)
    self.conv15 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm15 = nn.BatchNorm2d(512)
    self.leaky_relu15 = nn.LeakyReLU(negative_slope=0.1)
    self.conv16 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm16 = nn.BatchNorm2d(1024)
    self.leaky_relu16 = nn.LeakyReLU(negative_slope=0.1)
    self.conv17 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
    self.batchnorm17 = nn.BatchNorm2d(512)
    self.leaky_relu17 = nn.LeakyReLU(negative_slope=0.1)
    self.conv18 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
    self.batchnorm18 = nn.BatchNorm2d(1024)
    self.leaky_relu18 = nn.LeakyReLU(negative_slope=0.1)

    self.conv19 = nn.Conv2d(in_channels=1024, out_channels=num_classes, kernel_size=1, stride=1, padding=1, bias=False)
    self.avgpool= nn.AdaptiveAvgPool2d((1,1))
    self.max_pool2d=nn.MaxPool2d(2, stride=2)

  def forward(self, x):
    out=self.batchnorm1(self.conv1(x))
    out=self.leaky_relu1(out)
    out=self.max_pool2d(out)

    out=self.batchnorm2(self.conv2(out))
    out=self.leaky_relu2(out)
    out=self.max_pool2d(out)

    out=self.batchnorm3(self.conv3(out))
    out=self.leaky_relu3(out)
    out=self.batchnorm4(self.conv4(out))
    out=self.leaky_relu4(out)
    out=self.batchnorm5(self.conv5(out))
    out=self.leaky_relu5(out)
    out=self.max_pool2d(out)

    out=self.batchnorm6(self.conv6(out))
    out=self.leaky_relu6(out)
    out=self.batchnorm7(self.conv7(out))
    out=self.leaky_relu7(out)
    out=self.batchnorm8(self.conv8(out))
    out=self.leaky_relu8(out)
    out=self.max_pool2d(out)

    out=self.batchnorm9(self.conv9(out))
    out=self.leaky_relu9(out)
    out=self.batchnorm10(self.conv10(out))
    out=self.leaky_relu10(out)
    out=self.batchnorm11(self.conv11(out))
    out=self.leaky_relu11(out)
    out=self.batchnorm12(self.conv12(out))
    out=self.leaky_relu12(out)
    out=self.batchnorm13(self.conv13(out))
    out=self.leaky_relu13(out)
    out=self.max_pool2d(out)

    out=self.batchnorm14(self.conv14(out))
    out=self.leaky_relu14(out)
    out=self.batchnorm15(self.conv15(out))
    out=self.leaky_relu15(out)
    out=self.batchnorm16(self.conv16(out))
    out=self.leaky_relu16(out)
    out=self.batchnorm17(self.conv17(out))
    out=self.leaky_relu17(out)
    out=self.batchnorm18(self.conv18(out))
    out=self.leaky_relu18(out)

    out=self.conv19(out)
    out=self.avgpool(out)
    out=torch.flatten(out,1)

    return out

In [ ]:
class NewModel(nn.Module):
    def __init__(self, weight_path, output_layers, *args):
        super().__init__(*args)
        self.output_layers = output_layers
        self.selected_out = OrderedDict()
        self.pretrained = Darknet19()
        self.pretrained.load_state_dict(torch.load(weight_path))
        self.fhooks = []

        for i,l in enumerate(list(self.pretrained._modules.keys())):
            if i in self.output_layers:
                self.fhooks.append(getattr(self.pretrained,l).register_forward_hook(self.forward_hook(1)))

    def forward_hook(self, layer_name):
        def hook(module, input, output):
            self.selected_out[layer_name] = output
        return hook            
    
    def forward(self, x):
        out = self.pretrained(x)
        return out, self.selected_out

### **Build Model**

In [ ]:
weight_path = './darknet19_quan_weight1.pth'
model = NewModel(weight_path, output_layers = [i for i in range(60)]).to('cuda:0')

### **Load Dataset**

In [ ]:
data_path = "\wrk\xsjhdnobkup5\taeheej\dataset\imagenet"
traindir = os.path.join(data_path, 'train')
valdir = os.path.join(data_path, 'val3k')

normalize = transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1,1,1]) #scale[0,1]--->normalization unpreceded

batch_size=32
num_workers=16

train_transform = transforms.Compose([
      transforms.RandomResizedCrop(224), #무작위로 이미지를 크롭(잘라내고)하고 224*224 pixel로 크기 조정
      transforms.RandomHorizontalFlip(), #이미지를 수평 방향으로 무작위로 뒤집기-데이터 다양성을 증가. 이미지 방향에 덜 민감
      transforms.ToTensor(), #이미지를 pytorch 텐서(pytorch 모델에서 처리할 수 있는 데이터 형식)로 변환. 이미지 데이터는 0에서 255 범위의 정수에서 0.0에서 1.0 범위의 부동소수점으로 스케일링
      normalize,
    ])

valid_transform=transforms.Compose([
      transforms.Resize(256), #모든 이미지의 크기를 256x256 픽셀로 조절한다.
      transforms.CenterCrop(224), #중앙을 기준으로 224x224로 크롭하여 크기 조정
      transforms.ToTensor(), #이미지를 pytorch 텐서로 변환
      normalize
    ])

train_dataset=datasets.ImageFolder(traindir, train_transform) #traindir 이미지를 가져와서 train_transform 전처리를 거치고 이미지 데이터셋을 생성한다.

valid_dataset=datasets.ImageFolder(valdir, valid_transform)

train_loader=torch.utils.data.DataLoader( #데이터 로드 및 처리
        train_dataset, #사용될 데이터셋
        batch_size=batch_size, #모델에 한번에 공급될 샘플수
        shuffle=True, #각 에프크(epoch) 시작시 데이터셋을 무작위로 섞어서 데이터의 순서에 의존하지 않도록 한다. overfitting을 방지한다.
        num_workers=num_workers,
        pin_memory=True) #Dataloader가 GPU에 복사하기 전에 CPU의 고정된 메모리 영역(pin된 메모리)에 로드하도록 한다. 이는 CPU에서 GPU로의 데이터 전송 속도를 향상시켜 학습 성능을 높인다.

valid_loader=torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True)

print("number of training dataset:%d" % len(train_dataset))
print("number of validation dataset:%d" % len(valid_dataset))

### **collect p for each layer for 100 samples after weight quantization**

In [ ]:
for data, target in valid_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    out, layerout = model(data)
    break

count = 0
for k in layerout.keys():
    if k.startswith('leaky'):
        count += 1

quant_p = np.zeros((100, count+1))
batch = 0
for data, target in valid_loader:
    if train_on_gpu:
        data, traget = data.cuda(), target.cuda()
    out, layerout = model(data)
    col = 0
    for k in layerout.keys():
        if k.startswith('leaky') or k=='conv19':
            if batch == 0:
                print(k)
            _,p = quantize(layerout[k].cpu().detach().numpy())
            quant_p[batch, col] = p
            col += 1
    batch += 1
    if batch == 100:
        break

In [ ]:
#find most frequent clipping range
def mode(X):
    # X: List
    return max(set(X), key=X.count)

#create Lookup table
cfg = []
for i in range(quant_p.shape[1]):
    print(i, mode(list(quant_p[:,i])))
    cfg.append(int(mode(list(quant_p[:,i]))))


In [ ]:
class Darknet19q(nn.Module): # formal inheritance method in PyTorch / inherit Parent class nn.Module
    def __init__(self, p_list, num_classes: int = 1000): # 1000 is number of classes for neuralnet model to predict. It is default. you can reset it when calling the function
        super(Darknet19q, self).__init__() # super(child class name, self).__init__()

        self.p_list = p_list
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False) #3*3 require zero padding / 1*1 does not require zero padding
        self.batchnorm1 = nn.BatchNorm2d(32) #32개 channel 출력값을 normalization, Z function(평균, 분산)-->(0,1) gradient vanishing, exploding을 방지
        self.leaky_relu1 = nn.LeakyReLU(negative_slope=0.1) #Relu negative slope

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.leaky_relu2 = nn.LeakyReLU(negative_slope=0.1)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.leaky_relu3 = nn.LeakyReLU(negative_slope=0.1)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm4 = nn.BatchNorm2d(64)
        self.leaky_relu4 = nn.LeakyReLU(negative_slope=0.1)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm5 = nn.BatchNorm2d(128)
        self.leaky_relu5 = nn.LeakyReLU(negative_slope=0.1)

        self.conv6 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm6 = nn.BatchNorm2d(256)
        self.leaky_relu6 = nn.LeakyReLU(negative_slope=0.1)
        self.conv7 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm7 = nn.BatchNorm2d(128)
        self.leaky_relu7 = nn.LeakyReLU(negative_slope=0.1)
        self.conv8 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm8 = nn.BatchNorm2d(256)
        self.leaky_relu8 = nn.LeakyReLU(negative_slope=0.1)

        self.conv9 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm9 = nn.BatchNorm2d(512)
        self.leaky_relu9 = nn.LeakyReLU(negative_slope=0.1)
        self.conv10 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm10 = nn.BatchNorm2d(256)
        self.leaky_relu10 = nn.LeakyReLU(negative_slope=0.1)
        self.conv11 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm11 = nn.BatchNorm2d(512)
        self.leaky_relu11 = nn.LeakyReLU(negative_slope=0.1)
        self.conv12 = nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm12 = nn.BatchNorm2d(256)
        self.leaky_relu12 = nn.LeakyReLU(negative_slope=0.1)
        self.conv13 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm13 = nn.BatchNorm2d(512)
        self.leaky_relu13 = nn.LeakyReLU(negative_slope=0.1)

        self.conv14 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm14 = nn.BatchNorm2d(1024)
        self.leaky_relu14 = nn.LeakyReLU(negative_slope=0.1)
        self.conv15 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm15 = nn.BatchNorm2d(512)
        self.leaky_relu15 = nn.LeakyReLU(negative_slope=0.1)
        self.conv16 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm16 = nn.BatchNorm2d(1024)
        self.leaky_relu16 = nn.LeakyReLU(negative_slope=0.1)
        self.conv17 = nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False)
        self.batchnorm17 = nn.BatchNorm2d(512)
        self.leaky_relu17 = nn.LeakyReLU(negative_slope=0.1)
        self.conv18 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False)
        self.batchnorm18 = nn.BatchNorm2d(1024)
        self.leaky_relu18 = nn.LeakyReLU(negative_slope=0.1)

        self.conv19 = nn.Conv2d(in_channels=1024, out_channels=num_classes, kernel_size=1, stride=1, padding=1, bias=False)
        self.avgpool= nn.AdaptiveAvgPool2d((1,1))
        self.max_pool2d=nn.MaxPool2d(2, stride=2)
    
    def quantize(self, X, p=0, NBIT=8):
        QRANGE = 2**(NBIT-1)
        p = torch.tensor(p,dtype=torch.int8).to('cuda:0')
        data_th = torch.clamp(X, -QRANGE*torch.float_power(2, -p),(QRANGE-1)*torch.float_power(2, -p))

        SCALE = torch.float_power(2, -p)

        data_qn = torch.round(data_th/SCALE)

        data_dqn = data_qn*SCALE
        return data_dqn
    
    def forward(self, x):
        out=self.batchnorm1(self.conv1(x))
        out=self.leaky_relu1(out)
        out=self.quantize(out, p=self.p_list[0])
        out=self.max_pool2d(out)
    
        out=self.batchnorm2(self.conv2(out))
        out=self.leaky_relu2(out)
        out=self.quantize(out, p=self.p_list[1])
        out=self.max_pool2d(out)
    
        out=self.batchnorm3(self.conv3(out))
        out=self.leaky_relu3(out)
        out=self.quantize(out, p=self.p_list[2])
        out=self.batchnorm4(self.conv4(out))
        out=self.leaky_relu4(out)
        out = self.quantize(out, p=self.p_list[3])
        out=self.batchnorm5(self.conv5(x))
        out=self.leaky_relu5(out)
        out=self.quantize(out, p=self.p_list[4])
        out=self.max_pool2d(out)
    
        out=self.batchnorm6(self.conv6(x))
        out=self.leaky_relu6(out)
        out=self.quantize(out, p=self.p_list[5])
        out=self.batchnorm7(self.conv7(x))
        out=self.leaky_relu7(out)
        out=self.quantize(out, p=self.p_list[6])
        out=self.batchnorm8(self.conv8(x))
        out=self.leaky_relu8(out)
        out=self.quantize(out, p=self.p_list[7])
        out=self.max_pool2d(out)
    
        out=self.batchnorm9(self.conv9(x))
        out=self.leaky_relu9(out)
        out=self.quantize(out, p=self.p_list[8])
        out=self.batchnorm10(self.conv10(x))
        out=self.leaky_relu10(out)
        out=self.quantize(out, p=self.p_list[9])
        out=self.batchnorm11(self.conv11(x))
        out=self.leaky_relu11(out)
        out=self.quantize(out, p=self.p_list[10])
        out=self.batchnorm12(self.conv12(x))
        out=self.leaky_relu12(out)
        out=self.quantize(out, p=self.p_list[11])
        out=self.batchnorm13(self.conv13(x))
        out=self.leaky_relu13(out)
        out=self.quantize(out, p=self.p_list[12])
        out=self.max_pool2d(out)
    
        out=self.batchnorm14(self.conv14(x))
        out=self.leaky_relu14(out)
        out=self.quantize(out, p=self.p_list[13])
        out=self.batchnorm15(self.conv15(x))
        out=self.leaky_relu15(out)
        out=self.quantize(out, p=self.p_list[14])
        out=self.batchnorm16(self.conv16(x))
        out=self.leaky_relu16(out)
        out=self.quantize(out, p=self.p_list[15])
        out=self.batchnorm17(self.conv17(x))
        out=self.leaky_relu17(out)
        out=self.quantize(out, p=self.p_list[16])
        out=self.batchnorm18(self.conv18(x))
        out=self.leaky_relu18(out)
        out=self.quantize(out, p=self.p_list[17])
    
        out=self.conv19(out)
        out=self.quantize(out, p=self.p_list[18])
        out=self.avgpool(out)
        out=torch.flatten(out,1)
    
        return out

### **Accuracy of dynamic quantized model**

In [ ]:
model = Darknet19q()
#print(model)

#weight_path = "C:\Bigdata\"quantization\darknet19_224d.pth"
#model.Load_state_dict(torch.Load(weigtht_path, map_Location=torch.device('cpu')))

weight_path = './darknet19_224d.pth'
model.load_state_dict(torch.load(weight_path))

In [ ]:
model.eval() #model.eval(): 모델을 평가 모드로 설정한다. 
             #이는 모델이 학습(training) 모드가 아닌, 평가(evaluation) 모드로 실행되어야 할 때 사용된다. 
             #평가 모드에서는 모델의 행동이 바뀌는 일부 레이어들(예를 들어, Dropout, BatchNorm 등)이 평가 상황에 적합하게 동작하도록 조정된다. 
             #예를 들어, Dropout 레이어는 학습 동안에는 일부 뉴런을 무작위로 비활성화하지만, 평가 모드에서는 모든 뉴런을 활성화 상태로 유지한다.

model.cuda() #model.cuda(): 모델의 매개변수와 버퍼를 CUDA를 사용 가능한 GPU 메모리로 이동시킵니다. 
             #이를 통해 모델이 GPU에서 실행될 수 있게 하며, GPU의 병렬 처리 능력을 활용하여 더 빠른 계산이 가능해집니다. 
             #이 메소드를 호출하기 전에는 모델이 CPU 메모리에 있었을 것이고, 이 호출을 통해 모델의 연산이 GPU에서 수행되도록 변경됩니다.

top1 = AverageMeter('Acc@1', ':6.4f') #batch 마다 평균을 구해서 그 정확도를 계산한다. 총 6자리를 표현하고 소수점 넷째자리까지 표시한다. Fixed point 형식으로 나타낸다
                                      #3.141592 --> 3.1416
top5 = AverageMeter('Acc@5', ':6.4f')
for data, target in valid_loader: #배치 단위로 data와 target을 순회 
    if train_on_gpu:
        data, target = data.cuda(), target.cuda() #data와 target을 GPU로 이동
    #forward pass: compute predicted outputs by passing inputs to the model
    output = model(data) #model에 data 전달
    acc1, acc5 = accuracy(output, target, topk=(1, 5)) #Top-1과 Top-5 정확도를 계산
    top1.update(acc1.item(), target.size(0)) #acc1.item()은 Top-1 정확도의 스칼라 값을 반환, target.size(0)는 현재 배치의 크기를 의미
    top5.update(acc5.item(), target.size(0)) 

##top1 accuracy, top5 accuracy
print(top1, top5)    

### **현재(향후) 연구 방향**

### **Sensitivity Analysis and Mixed Precision**

딥러닝 모델의 모든 층이 양자화에 똑같이 반응하는 것은 아니다.

어떤 층은 presicion 저하에 좀 더 sensitive하고, 다른 층은 non-sensitive할 수 있다.

따라서 각 층의 양자화에 대한 민감도(sensitivity)를 측정해서 그 민감도에 따라 다른 bit의 quantization을 구현할 수 있다.

예를 들어 32bit, 16bit, 8bit, 4bit, 2bit등을 각 층에 따라 다르게 적용할 수 있다.

sensitivity를 측정하는 방법과 mixed precision을 지원하는 하드웨어를 개발하는 것은 현재의 연구 방향이다. 



### **Quantization-Aware Training(QAT)

부동수소점 32bit에서 8bit로의 변환에 따른 정밀도의 손실로 인해서 Post-Training quantization된 딥러닝 모델의 accuracy는 감소할 가능성이 크다.

Quantization-Aware 양자화는 이러한 양자화에 따른 손실을 딥러닝 모델을 훈현시킬 때에 포함시키는 것이다.

이를 위한 여러가지 방법들이 시도되고 있다.

한가지 예로는 고정소수점 8비트로 모델을 재훈련시키는 것이다.

모델의 모든 가중치는 32bit로 저장된다.

Backward Propagation은 통상적인 방법으로 진행된다.

그러나 forward pass에서 모델을 quantization시키고, 이에 따른 정확도를 기준으로 모델을 finetuning하게 된다.